In [ ]:
import pandas as pd

In [ ]:
import os
import sys
os.chdir('..')
sys.path.append('src')

In [ ]:
import utils
import rle2tiff

In [ ]:
imgs = utils.get_filenames('input/hm/test', '*.tiff', lambda x: False)
imgs

In [ ]:
hws = {}
full_names = {}
for f in imgs:
    _, hw, _ = utils.get_basics_rasterio(str(f))
    hws[f.stem] = hw
    full_names[f.stem] = f
hws

In [ ]:
df_train = pd.read_csv('input/hm/HuBMAP-20-dataset_information.csv')

In [ ]:
t = df_train[['image_file','width_pixels','height_pixels']]
sanity = {a.split('.')[0]:(c,b) for a,b,c in t.values}
sanity

In [ ]:
for k,v in sanity.items():
    if k in hws:
        if hws[k] != v:
            print('ERR',k,v,hws[k])
        else:
            print('OKK',k,v,hws[k])
            

In [ ]:
df = pd.read_csv('/home/gsm/test/submission_86.csv')

In [ ]:
d = df.to_dict()
ids, preds = d['id'], d['predicted']

In [ ]:
dd = {}
for k in ids:
    dd[ids[k]] = preds[k]

In [ ]:
for k, rle in dd.items():
    hw = hws[k]
    print(k, hw)
    mask  = rle2tiff.rle2mask(rle, hw[::-1])
    mask_name =  f'output/predict_86/masks/{k}_86.tiff'
    #utils.save_tiff_uint8_single_band(mask.T, mask_name)
    #merge_name = f'output/predict_86/merged/{k}_86.tiff'
    #utils.tiff_merge_mask(full_names[k], mask_name, merge_name)
    

In [ ]:
mask.T.shape

In [ ]:
import rasterio as rio
from callbacks import dice_loss

In [ ]:
import torch

In [ ]:
for k, rle in dd.items():
    hw = hws[k]
    print(k, hw)
    mask  = rle2tiff.rle2mask(rle, hw[::-1])
    mymask =  rio.open(f'output/predicts/masks/{k}.tiff').read()
    dice = dice_loss(torch.from_numpy(mask.T/255), torch.from_numpy(mymask))
    print(k, dice)

In [ ]:
for k, rle in dd.items():
    hw = hws[k]
    print(k, hw)
    mask_name1 =  f'output/predict_86/masks/{k}_86.tiff'
    mask_name2 =  f'output/predicts/masks/{k}.tiff'
    merge_name = f'output/predict_86/combined/{k}.tiff'
    
    utils.tiff_merge_mask(full_names[k], mask_name1, merge_name, mask_name2)
    #break